In [2]:
require(data.table)
require(tidyverse)

## 1. Read data 

In [3]:
dat.mat <- fread("gzcat ../dat/1910_v2//long_matrix_w_transcripts.txt.gz") %>% filter(is.na(tr.idx))
dim(dat.mat)
head(dat.mat)


[1] 24864376        7

seq,start,end,cell,value,tr.idx,gene
chr10,100005281,100005858,Islet1fresh_AGACACCTCGATCAGTACTGCATAGGCTC,2,NA,
chr10,100005281,100005858,Islet1fresh_AGACACCTCTCTCTACGTAAGGAGAGGCG,2,NA,
chr10,100005281,100005858,Islet1fresh_AGACACCTCTCTCTACGTAAGGAGGGCTC,2,NA,
chr10,100005281,100005858,Islet1fresh_AGCGATAGTCCTGAGCTTATGCGAGTACT,1,NA,
chr10,100005281,100005858,Islet1fresh_ATTACTCGATCTCAGGAAGGAGTAGTACT,2,NA,
chr10,100005281,100005858,Islet1fresh_ATTACTCGCTCTGGTAGTAAGGAGCCTAT,1,NA,


 Concepts: 
1. `promoter region`: -500bp + 500bp for all TSS in gencode 
2. `promoter Peaks`: peaks that overlap promoter region

In [6]:
dat.mat <- dat.mat %>% mutate(seq = sub("chr", "", seq)) %>% mutate(distal_peak = paste0(seq, 
    ":", start, "-", end)) %>% select(distal_peak, cell) %>% distinct()
dim(dat.mat)

[1] 24864376        2

In [9]:
dat.mat %>%head(1)
dat.mat <-dat.mat %>% left_join(fread("../dat/1910_v2/islet.cluster_labels.filt.txt") %>% select(index, 
    cluster_name), by = c(cell = "index"))
dat.mat %>%head(1)
dat.mat %>%dim

distal_peak,cell
10:100005281-100005858,Islet1fresh_AGACACCTCGATCAGTACTGCATAGGCTC


distal_peak,cell,cluster_name
10:100005281-100005858,Islet1fresh_AGACACCTCGATCAGTACTGCATAGGCTC,alpha_1


[1] 24864376        3

In [27]:
# get all cells 
dat.all.cells <- table((fread('../dat/1910_v2/islet.cluster_labels.filt.txt'))$cluster) # table for all cells
dat.all.cells
sum(dat.all.cells)



     acinar     alpha_1     alpha_2      beta_1      beta_2     delta_1 
         46        5191        1027        4204        3394         478 
    delta_2      ductal endothelial       gamma      immune    stellate 
        232          80         118         260         140         128 

[1] 15298

In [24]:
sub('(_1)|(_2)','',names(dat.all.cells))

[1] "acinar"      "alpha"       "alpha"       "beta"        "beta"       
 [6] "delta"       "delta"       "ductal"      "endothelial" "gamma"      
[11] "immune"      "stellate"

In [25]:
sum(dat.all.cells)

[1] 3298

In [14]:
# how many peaks
length(unique(dat.mat$distal_peak))

[1] 209947

In [16]:
dat.mat <- dat.mat%>%rename(cluster=cluster_name)

## 2 Prepare data


Peak|n_cells|celltype 
--- | --- | ---


In [19]:
# prepare data
setDT(dat.mat)
dat.mat<- dat.mat[,.N,by=.(distal_peak,cluster)]%>%arrange(distal_peak,cluster)

In [20]:
head(dat.mat,1)

distal_peak,cluster,N
1:100009712-100010377,alpha_1,28


## 4. Fisher's exact test  between subtypes

As long as there is any  peaks in that cell, the peak is open in this cell. 

1. get total alpha 1 and alpha 2 cells 
2. test hit in alpha1 vs hit in alpha2  (create contentigen table)
3. perform [Fisher's exact test](https://en.wikipedia.org/wiki/Fisher%27s_exact_test) or [chi-squared test](https://en.wikipedia.org/wiki/Chi-squared_test)

Input: `dat.mat` and `dat.all.cells`
output: 

peak|n_celltype1|n_celltype2|total_cells|celltype | pval | frac_1 | frac_2| odds


### 4.1 perform fisher's exact test for distal peaks

In [28]:
tr = "1:100009712-100010377"
celltypes <- c("alpha_1", "alpha_2")

test.dat <- dat.mat %>% filter(distal_peak == tr & cluster %in% celltypes)
test.dat
table.res<- test.dat$N; names(table.res) <- test.dat$cluster
# handle if 0 for one subtype
a = setdiff(celltypes, names(table.res))
table.res[a] <- 0
test.tab <- matrix(c(table.res[celltypes[1]], table.res[celltypes[2]], dat.all.cells[celltypes[1]] - 
    table.res[celltypes[1]], dat.all.cells[celltypes[2]] - table.res[celltypes[2]]), 
    byrow = T, nrow = 2, dimnames = list(expressed = c("Yes", "No"), subtype = celltypes))
test.tab
test.tab[1]
test.tab[2]


f1 <- test.tab[1]/(test.tab[1] + test.tab[2])
f2 <- test.tab[3]/(test.tab[3] + test.tab[4])
l <- ifelse(f1 > f2, "greater", "less")
test.res <- fisher.test(test.tab, alternative = l)
res <- list(pval = test.res$p.value, odds = test.res$estimate, type1_frac = f1, type2_frac = f2)
res

fisher.test(test.tab)

distal_peak,cluster,N
1:100009712-100010377,alpha_1,28
1:100009712-100010377,alpha_2,4


,alpha_1,alpha_2
Yes,28,4
No,5163,1023


[1] 28

[1] 5163

$pval
[1] 0.3721211

$odds
odds ratio 
  1.386916 

$type1_frac
[1] 0.005393951

$type2_frac
[1] 0.003894839


	Fisher's Exact Test for Count Data

data:  test.tab
p-value = 0.8104
alternative hypothesis: true odds ratio is not equal to 1
95 percent confidence interval:
 0.4837113 5.4531250
sample estimates:
odds ratio 
  1.386916 


In [30]:

fun.ftestPerTr <- function(  tr='1:100009712-100010377',#=1
                             celltypes=c('alpha_1','alpha_2'),
                             dat=dat.mat){
    

test.dat <- dat %>% filter(distal_peak == tr & cluster %in% celltypes)
 table.res<- test.dat$N; names(table.res) <- test.dat$cluster

    # handle if 0 for one subtype 
    a=setdiff(celltypes,    names(table.res))
    table.res[a]<-0
    
    test.tab <- matrix(c(table.res[celltypes[1]], table.res[celltypes[2]], 
                         dat.all.cells[celltypes[1]]-table.res[celltypes[1]], 
                         dat.all.cells[celltypes[2]]-table.res[celltypes[2]]),
                       byrow =  T,       
                       nrow = 2,
                       dimnames = list(expressed = c("Yes", "No"),
                       subtype = celltypes))

    f1<-test.tab[1]/(test.tab[1]+test.tab[2])
    f2 <- test.tab[3]/(test.tab[3]+test.tab[4])
    l <-ifelse(f1>f2,'greater','less')
    test.res <- fisher.test(test.tab,alternative = l)
    res <- list(
                distal_peak =tr,
                pval=test.res$p.value,
                odds=test.res$estimate,
                type1_frac=f1,
                type2_frac=f2
               )
    res
}

##

celltypes <- c('alpha_1','alpha_2')

system.time(fun.ftestPerTr(celltypes=celltypes))
t(fun.ftestPerTr(celltypes=celltypes))


   user  system elapsed 
  0.068   0.003   0.071 

distal_peak,pval,odds,type1_frac,type2_frac
1:100009712-100010377,0.3721211,1.386916,0.005393951,0.003894839


In [31]:

celltypes <- c('alpha_1','alpha_2')
dat.mat.sub <- dat.mat%>%filter(cluster %in% celltypes)
fun.ftestPerTr <- function(  tr='1:100009712-100010377',#=1
                             celltypes=c('alpha_1','alpha_2'),
                             dat=dat.mat.sub){
    

test.dat <- dat %>% filter(distal_peak == tr)
 table.res<- test.dat$N; names(table.res) <- test.dat$cluster

    # handle if 0 for one subtype 
    a=setdiff(celltypes,    names(table.res))
    table.res[a]<-0
    
    test.tab <- matrix(c(table.res[celltypes[1]], table.res[celltypes[2]], 
                         dat.all.cells[celltypes[1]]-table.res[celltypes[1]], 
                         dat.all.cells[celltypes[2]]-table.res[celltypes[2]]),
                       byrow =  T,       
                       nrow = 2,
                       dimnames = list(expressed = c("Yes", "No"),
                       subtype = celltypes))

    f1<-test.tab[1]/(test.tab[1]+test.tab[2])
    f2 <- test.tab[3]/(test.tab[3]+test.tab[4])
    l <-ifelse(f1>f2,'greater','less')
    test.res <- fisher.test(test.tab,alternative = l)
    res <- list(
                distal_peak =tr,
                pval=test.res$p.value,
                odds=test.res$estimate,
                type1_frac=f1,
                type2_frac=f2
               )
    res
}

##



system.time(fun.ftestPerTr(celltypes=celltypes))
t(fun.ftestPerTr(celltypes=celltypes))


   user  system elapsed 
  0.025   0.001   0.026 

distal_peak,pval,odds,type1_frac,type2_frac
1:100009712-100010377,0.3721211,1.386916,0.005393951,0.003894839


In [32]:

# estimate time
celltypes <- list()
celltypes$alpha <- c("alpha_1", "alpha_2")
celltypes$beta <- c("beta_1", "beta_2")
celltypes$delta <- c("delta_1", "delta_2")
sum(sapply( c("alpha", "beta", "delta"),function(x) 
    length(unique((dat.mat%>% filter(cluster %in% celltypes[[x]]))$distal_peak))))*.029/3600

[1] 4.513649

In [33]:
celltypes <- list()
celltypes$alpha <- c("alpha_1", "alpha_2")
celltypes$beta <- c("beta_1", "beta_2")
celltypes$delta <- c("delta_1", "delta_2")

In [34]:
## test run

x<- "beta"
dat.mat.sub <- dat.mat%>%filter(cluster %in% celltypes[[x]])
all.tr <- unique(dat.mat.sub$distal_peak)
length(all.tr)
do.call(rbind,lapply(all.tr[1:10],function(trr) t(fun.ftestPerTr(dat = dat.mat.sub, 
        tr = trr, celltypes = celltypes[[x]]))))

[1] 204254

distal_peak,pval,odds,type1_frac,type2_frac
1:100009712-100010377,0.2891137,0.9213313,0.03020932,0.03270477
1:100014525-100015253,0.05388995,1.304241,0.02521408,0.01944608
1:100017588-100018137,4.131075e-06,0.1485343,0.001189343,0.007955215
1:100023302-100023931,0.003786372,0.3216857,0.00190295,0.005892752
1:100056228-100056811,0.008416978,0.6015636,0.009990485,0.01649971
1:100064582-100064908,0.6938878,0.8073071,0.0002378687,0.0002946376
1:100065306-100065506,0.4203372,0.4035669,0.0002378687,0.0005892752
1:10010367-10010795,0.02110914,1.520098,0.01736441,0.01149087
1:100113698-100114049,0.0002234783,1.671828,0.03449096,0.02091927
1:100122820-100123020,0.4466965,0,0,0.0002946376


In [95]:
require(parallel)

res.transcript_level <- list()
# time consumming task
system.time(for (x in c("alpha", "beta", "delta")) {
    
    dat.mat.sub <- dat.mat %>% filter(cluster %in% celltypes[[x]])
    all.tr <- unique(dat.mat.sub$distal_peak)
    
    res.transcript_level[[x]] <- do.call(rbind, mclapply(all.tr, function(trr) t(fun.ftestPerTr(dat = dat.mat.sub, 
        tr = trr, celltypes = celltypes[[x]])), mc.cores = 10))
})

saveRDS(object = res.transcript_level, file = "../dat/1901/res.distal.peaks.fisher.Rds")

    user   system  elapsed 
8810.949  620.087 3677.038 

In [100]:
res.transcript_level <- rbind(data.frame(res.transcript_level$alpha,celltype="alpha"),
     data.frame(res.transcript_level$beta,celltype="beta"),
     data.frame(res.transcript_level$delta,celltype="delta"))


In [104]:
head(res.transcript_level)
dim(res.transcript_level)[1]/3600*.028

distal_peak,pval,odds,type1_frac,type2_frac,celltype
1_100009936_100010354,0.4746936,0.881426,0.003978469,0.004511278,alpha
1_100014517_100015228,0.5323403,1.089823,0.00491458,0.004511278,alpha
1_100017588_100018128,0.001922056,0.1549027,0.0009361105,0.006015038,alpha
1_100023318_100023949,0.0004998282,0.1547046,0.001170138,0.007518797,alpha
1_100056203_100056908,0.476826,1.868512,0.001404166,0.0007518797,alpha
1_100064607_100064883,0.7626272,Inf,0.0002340276,0,alpha


[1] 4.481439

In [105]:
fwrite(res.transcript_level, file = "../dat/1910_v2/res.distal.peaks.fisher.cvs")